# Testing
### Eheschliessung nach Nationalität der Eheschliessenden, seit 1993
Datum: 02.11.2021

**Importiere die notwendigen Packages**

In [7]:
#pip install geopandas fiona requests folium mplleaflet contextily

In [67]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [68]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_ehe_nationalitaet_od3305

(int_dwh)

In [69]:
# Filepath
fp = r"//szh.loc/ssz/applikationen/OGD_Dropzone/INT_DWH/bev_ehe_nationalitaet_OD3305/BEV330OD3305.csv"


# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )


In der Folge ein paar erste Tests:

In [70]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [71]:
data2betested.head(3)

,StichtagDatJahr,NationFHistLang,NationFHistSort,NationMHistLang,NationMHistSort,RegionFLang,RegionFSort,KontinentFLang,KontinentFSort,RegionMLang,RegionMSort,KontinentMLang,KontinentMSort,AnzSchl
0,1993,Schweiz,8100,Schweiz,8100,Zentraleuropa,13,Europa,1,Zentraleuropa,13,Europa,1,1143
1,1993,Schweiz,8100,Albanien,8201,Zentraleuropa,13,Europa,1,Südosteuropa,17,Europa,1,3
2,1993,Schweiz,8100,Belgien,8204,Zentraleuropa,13,Europa,1,Westeuropa,14,Europa,1,1


In [72]:
data2betested.dtypes

StichtagDatJahr     int64
NationFHistLang    object
NationFHistSort     int64
NationMHistLang    object
NationMHistSort     int64
RegionFLang        object
RegionFSort         int64
KontinentFLang     object
KontinentFSort      int64
RegionMLang        object
RegionMSort         int64
KontinentMLang     object
KontinentMSort      int64
AnzSchl             int64
dtype: object

In [73]:
data2betested.shape

(14007, 14)

Beschreibe einzelne Attribute

In [74]:
data2betested.describe()

,StichtagDatJahr,NationFHistSort,NationMHistSort,RegionFSort,KontinentFSort,RegionMSort,KontinentMSort,AnzSchl
count,14007,14007,14007,14007,14007,14007,14007,14007
mean,2007,8323,8303,25,2,24,2,7
std,8,209,194,19,2,17,2,46
min,1993,8100,8100,11,1,11,1,1
25%,2001,8212,8207,13,1,13,1,1
50%,2008,8249,8236,16,1,16,1,1
75%,2014,8423,8406,34,3,31,3,4
max,2020,8999,8999,88,8,88,8,1143


Wie viele Nullwerte gibt es im Datensatz?

In [75]:
data2betested.isnull().sum()

StichtagDatJahr    0
NationFHistLang    0
NationFHistSort    0
NationMHistLang    0
NationMHistSort    0
RegionFLang        0
RegionFSort        0
KontinentFLang     0
KontinentFSort     0
RegionMLang        0
RegionMSort        0
KontinentMLang     0
KontinentMSort     0
AnzSchl            0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [98]:
data2betested['StichtagDatJahr']=pd.to_datetime(data2betested['StichtagDatJahr'], format='%Y')

#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested

,StichtagDatJahr,NationFHistLang,NationFHistSort,NationMHistLang,NationMHistSort,RegionFLang,RegionFSort,KontinentFLang,KontinentFSort,RegionMLang,RegionMSort,KontinentMLang,KontinentMSort,AnzSchl
0,1993-01-01,Schweiz,8100,Schweiz,8100,Zentraleuropa,13,Europa,1,Zentraleuropa,13,Europa,1,1143
1,1993-01-01,Schweiz,8100,Albanien,8201,Zentraleuropa,13,Europa,1,Südosteuropa,17,Europa,1,3
2,1993-01-01,Schweiz,8100,Belgien,8204,Zentraleuropa,13,Europa,1,Westeuropa,14,Europa,1,1
3,1993-01-01,Schweiz,8100,Bulgarien,8205,Zentraleuropa,13,Europa,1,Südosteuropa,17,Europa,1,2
4,1993-01-01,Schweiz,8100,Dänemark,8206,Zentraleuropa,13,Europa,1,Nordeuropa,11,Europa,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14002,2020-01-01,Staat unbekannt,8999,Thailand,8542,Unzuteilbar,88,Unzuteilbar,8,Südostasien,47,Asien,4,1
14003,2020-01-01,Staat unbekannt,8999,Bangladesch,8546,Unzuteilbar,88,Unzuteilbar,8,Südasien,46,Asien,4,2
14004,2020-01-01,Staat unbekannt,8999,Armenien,8560,Unzuteilbar,88,Unzuteilbar,8,Südwestasien,45,Asien,4,1
14005,2020-01-01,Staat unbekannt,8999,Georgien,8562,Unzuteilbar,88,Unzuteilbar,8,Südwestasien,45,Asien,4,1


Bereite die Daten für die Grafik unten vor

In [99]:
Jahre = data2betested.groupby("StichtagDatJahr")
len(Jahre)

28

In [100]:
for key, group in Jahre:
    print(key, len(group))

1993-01-01 00:00:00 402
1994-01-01 00:00:00 399
1995-01-01 00:00:00 379
1996-01-01 00:00:00 384
1997-01-01 00:00:00 402
1998-01-01 00:00:00 421
1999-01-01 00:00:00 449
2000-01-01 00:00:00 459
2001-01-01 00:00:00 485
2002-01-01 00:00:00 513
2003-01-01 00:00:00 543
2004-01-01 00:00:00 493
2005-01-01 00:00:00 508
2006-01-01 00:00:00 544
2007-01-01 00:00:00 538
2008-01-01 00:00:00 527
2009-01-01 00:00:00 529
2010-01-01 00:00:00 546
2011-01-01 00:00:00 559
2012-01-01 00:00:00 559
2013-01-01 00:00:00 556
2014-01-01 00:00:00 560
2015-01-01 00:00:00 543
2016-01-01 00:00:00 524
2017-01-01 00:00:00 560
2018-01-01 00:00:00 564
2019-01-01 00:00:00 580
2020-01-01 00:00:00 481


In [101]:
myJahr = "2020-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr

,StichtagDatJahr,NationFHistLang,NationFHistSort,NationMHistLang,NationMHistSort,RegionFLang,RegionFSort,KontinentFLang,KontinentFSort,RegionMLang,RegionMSort,KontinentMLang,KontinentMSort,AnzSchl
13526,2020-01-01,Schweiz,8100,Schweiz,8100,Zentraleuropa,13,Europa,1,Zentraleuropa,13,Europa,1,923
13527,2020-01-01,Schweiz,8100,Albanien,8201,Zentraleuropa,13,Europa,1,Südosteuropa,17,Europa,1,2
13528,2020-01-01,Schweiz,8100,Belgien,8204,Zentraleuropa,13,Europa,1,Westeuropa,14,Europa,1,8
13529,2020-01-01,Schweiz,8100,Dänemark,8206,Zentraleuropa,13,Europa,1,Nordeuropa,11,Europa,1,4
13530,2020-01-01,Schweiz,8100,Deutschland,8207,Zentraleuropa,13,Europa,1,Zentraleuropa,13,Europa,1,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14002,2020-01-01,Staat unbekannt,8999,Thailand,8542,Unzuteilbar,88,Unzuteilbar,8,Südostasien,47,Asien,4,1
14003,2020-01-01,Staat unbekannt,8999,Bangladesch,8546,Unzuteilbar,88,Unzuteilbar,8,Südasien,46,Asien,4,2
14004,2020-01-01,Staat unbekannt,8999,Armenien,8560,Unzuteilbar,88,Unzuteilbar,8,Südwestasien,45,Asien,4,1
14005,2020-01-01,Staat unbekannt,8999,Georgien,8562,Unzuteilbar,88,Unzuteilbar,8,Südwestasien,45,Asien,4,1


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [102]:
#test = data2betested.groupby(['StichtagDatJahr', 'AlterV20Kurz'])[['AnzBestWir']].agg()

# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()
    


    

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [109]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [110]:
staatenlos_unbekannt = data2betested[(data2betested.NationFHistSort == 8998) | (data2betested.NationFHistSort == 8999)| (data2betested.NationMHistSort == 8998) | (data2betested.NationMHistSort == 8999)]
mySelection = staatenlos_unbekannt[(staatenlos_unbekannt.StichtagDatJahr >"2015-01-01")]
#mySelection = data2betested[(data2betested.StichtagDatJahr>"2011-01-01") & (data2betested.QuarSort<70)]
mySelection

,StichtagDatJahr,NationFHistLang,NationFHistSort,NationMHistLang,NationMHistSort,RegionFLang,RegionFSort,KontinentFLang,KontinentFSort,RegionMLang,RegionMSort,KontinentMLang,KontinentMSort,AnzSchl
11383,2016-01-01,Schweiz,8100,Staat unbekannt,8999,Zentraleuropa,13,Europa,1,Unzuteilbar,88,Unzuteilbar,8,4
11412,2016-01-01,Deutschland,8207,Staat unbekannt,8999,Zentraleuropa,13,Europa,1,Unzuteilbar,88,Unzuteilbar,8,29
11425,2016-01-01,Frankreich,8212,Staat unbekannt,8999,Westeuropa,14,Europa,1,Unzuteilbar,88,Unzuteilbar,8,2
11433,2016-01-01,Griechenland,8214,Staat unbekannt,8999,Südeuropa,16,Europa,1,Unzuteilbar,88,Unzuteilbar,8,3
11444,2016-01-01,Grossbritannien,8215,Staat unbekannt,8999,Westeuropa,14,Europa,1,Unzuteilbar,88,Unzuteilbar,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14002,2020-01-01,Staat unbekannt,8999,Thailand,8542,Unzuteilbar,88,Unzuteilbar,8,Südostasien,47,Asien,4,1
14003,2020-01-01,Staat unbekannt,8999,Bangladesch,8546,Unzuteilbar,88,Unzuteilbar,8,Südasien,46,Asien,4,2
14004,2020-01-01,Staat unbekannt,8999,Armenien,8560,Unzuteilbar,88,Unzuteilbar,8,Südwestasien,45,Asien,4,1
14005,2020-01-01,Staat unbekannt,8999,Georgien,8562,Unzuteilbar,88,Unzuteilbar,8,Südwestasien,45,Asien,4,1


In [112]:
# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()
    
myGrpSelectionSpecial = mySelection
myGrpSelectionSpecial \
    .groupby(['StichtagDatJahr', 'NationFHistLang', 'NationMHistLang' ]) \
    .agg(anz_Schl=('AnzSchl', 'sum')) \
    .query('StichtagDatJahr < "2018"') \
    .sort_values('anz_Schl', ascending=False) \
    .head(10)

anz_Schl
StichtagDatJahr NationFHistLang NationMHistLang          
2017-01-01      Staat unbekannt Deutschland            64
2016-01-01      Staat unbekannt Deutschland            58
2017-01-01      Deutschland     Staat unbekannt        39
2016-01-01      Deutschland     Staat unbekannt        29
                Staat unbekannt Indien                 28
2017-01-01      Staat unbekannt Italien                24
                                Indien                 23
2016-01-01      Staat unbekannt Portugal               18
2017-01-01      Staat unbekannt Portugal               17
2016-01-01      Staat unbekannt Italien                16

In [185]:
regions = data2betested.groupby(['RegionFLang', 'RegionFSort'])[['AnzSchl']].sum()
regions.sort_values('RegionFSort', ascending=True)

,,AnzSchl
RegionFLang,RegionFSort,
Nordeuropa,11,710
Osteuropa,12,1782
Zentraleuropa,13,55848
Westeuropa,14,2203
Südwesteuropa,15,2864
Südeuropa,16,4979
Südosteuropa,17,7572
Nordafrika,21,800
Ostafrika,22,519


In [212]:
mySel = data2betested[['StichtagDatJahr', 'RegionMLang', 'RegionMSort', 'RegionFLang', 'RegionFSort','AnzSchl']]
myGrpSelection = mySel\
    .groupby(['StichtagDatJahr', 'RegionMLang', 'RegionMSort', 'RegionFLang', 'RegionFSort']) \
    .agg(anz_Schl=('AnzSchl', 'sum')) \
    .query('StichtagDatJahr == "2020-01-01"') \
    .sort_values('anz_Schl', ascending=False) \
    .head(100)

myGrpSelection

anz_Schl
StichtagDatJahr RegionMLang        RegionMSort RegionFLang          RegionFSort          
2020-01-01      Zentraleuropa      13          Zentraleuropa        13               1358
                Westeuropa         14          Zentraleuropa        13                 68
                Zentraleuropa      13          Unzuteilbar          88                 68
                Südeuropa          16          Zentraleuropa        13                 63
                Zentraleuropa      13          Osteuropa            12                 59
...                                                                                   ...
                Südliches Ozeanien 56          Nordeuropa           11                  2
                Südostasien        47          Südostasien          47                  2
                                               Unzuteilbar          88                  2
                Südeuropa          16          Mittleres Südamerika 35                  2
                Südwesteuropa      15          Mittleres Südamerika 35                  2

[100 rows x 1 columns]

In [213]:
#myGrpSelection.to_csv("C:/Temp/test.csv")

In [214]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]
#mySelection[[mySelection.NationFHistLang =='Schweiz']]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['RegionFLang'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(myGrpSelection).encode(
    x='RegionMLang',
    y='AnzSchl',
    color='RegionFLang'
)

points = base.mark_circle().encode(
    opacity=alt.value(2)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

ValueError: Hierarchical indices not supported

alt.LayerChart(...)

In [215]:
alt.Chart(myGrpSelection).mark_circle(size=60).encode(
    x='AnzSchl',
    y='NationFHistLang',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='greens')),
    color=alt.Color('StichtagDatJahr', scale=alt.Scale(scheme='dark2')),
    tooltip=['AnzSchl','StichtagDatJahr','NationFHistLang','NationMHistLang']
).interactive() # this makes the axes interactive: now you can zoom & pan




ValueError: Hierarchical indices not supported

alt.Chart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=52&Source=%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs)**